# Backpropogation from Scratch 

#### Final Project for COMPSCIX404 - Data Structures and Algorithms
#### Nikhil Gopal 
#### August 2024

In this project, I will build the backpropogation algorithm, and show how it can be used to compute the gradient of the cost function when training a Neural Network with the Pytorch Deep Learning Framework. Neural Networks trained with backpropogation can be applied to text generation, computer vision, classification, facial recognition, audio generation, speech to text and many more applications.

I will start by explaining the algorithm and it's applicability to modern Deep Learning, with an overview of the mathematics behind it. Then, I will analyze the time and space complexity of the algorithm before presenting a code implementation.




## Intro and Motivation

Fundamentally, every problem in Machine Learning is an **optimization problem**. Optimization is the process of finding the maximum or minimum value of a function, such that it behaves optimally. In the case of Machine Learning, we seek to optimize the *loss function*, which describes the difference between the expected output of an ML model, and it's actual output. For example, if we were predicting human height and our model predicted 100 cm for a person who's true height was 105 cm, the loss would be $105-100=5$ cm. ML models have changable *parameters* whose optimal values can be learned from data such that the model's loss function is as close as possible to zero. 


Deep Learning is a subsect of Machine Learning that uses a specific type of Machine Learning model called a Multi-Layer Perceptron, also commonly referred to as Neural Networks. Neural Networks pass an input through different "layers" of the model, which all contribute to the model's final output. These models often have much higher numbers of learnable parameters than simpler models, but the computations corresponding to a single input must be performed sequentially and cannot be parallelized, since the input of a later layer must be the output of a previous layer:

**Input image depicting Neural Net here**

Many advancements in Machine Learning have come from using Deep Neural Networks (models with lots of layers and parameters) to learn the relevant features of and transform inputted data in novel ways. By leveraging techniques from Calculus, we can compute the gradient vector of a Neural Network's cost function. Each index of the gradient vector contains the partial derivative of the cost function with respect to a parameter, which describes the impact that each individual parameter has on the total loss of the network. The sum of all the loss that comes from every input parameter is the total loss of the network, thus making computing the gradient a key factor in minimizing the loss function.

Once the gradient is computed, the model's paramaters are adjusted using a parameter estimation method such as gradient descent, and the loss and parameters are continously adjusted and recomputed until the model's behavior more closely mimics the information it learned from the training data.

As model sizes keep increasing, an efficient algorithm is needed to compute the gradient. This algorithm should ideally be computationally efficient so as to minimize total training time and memory usage when training neural networks. This will ensure that models can be deployed into producition faster, and that they can be made more accuracte by being able to be trained on more data in less time.

## Introduction to backpropagation

Math explaining in detail here

## Pseudocode

## Code and Practical Example

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

At this step, we define transformations that will turn our image datasets into tensors that pytorch can use, and normalize them, with mean $0.5$ and standard deviation $0.5$, so that values are approximately between the range $[-1,1]$. This is optimal for the allowing the gradient descent algorithm to converge to a true local minimum, and would be very necessary if you were using input features with different scales (though we aren't here).

Next, we download training and test datasets, and apply our transformation onto them, and define our batch size. This helps improve computational efficiency by computing the gradient and loss of a batch of weights, and then adjusting the parameters based on a batch, instead of once for each image.

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


Next we'll define a Neural Network class, as a child class of PyTorch's nn.Module class. In the initialization, we'll first use the initialization of the parent function, and then create our layers. Each MNIST image is a 28x28 image. After applying the transformation, pytorch flattens this automatically  into a 1D 28*28 = 784 length vector in the input layer. 

The first hidden layer fc1 thus takes 28*28 input nodes, and outputs 128, which is the input size of the next hidden layer. We then reduce the dimensionality to 64 before

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

        
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [3]:
model = SimpleNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training the model
for epoch in range(5):  # Number of epochs
    running_loss = 0.0
    for inputs, labels in trainloader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

print('Finished Training')

# Testing the model
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total} %')

Epoch 1, Loss: 0.4227850303800503
Epoch 2, Loss: 0.18438721798670127
Epoch 3, Loss: 0.13313483049187547
Epoch 4, Loss: 0.10535326947384616
Epoch 5, Loss: 0.08892478920551124
Finished Training
Accuracy on the test set: 96.39 %
